In [2]:
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import sys,os
from PIL import Image
from PIL.ImageQt import ImageQt
from skimage import io, color,img_as_float,img_as_uint

import numpy as np

In [3]:
class ImageBox(QWidget):
    def __init__(self,px=None):
        super(ImageBox,self).__init__()
        self.px = px
    
    def paintEvent(self, paint_event):
        painter = QPainter(self)
        if self.px==None:
            painter.drawText(QPoint(self.width()//2-5,self.height()//2-5),"No Painting")
        else:
            painter.drawPixmap(self.rect(), self.px)
            painter.setRenderHint(QPainter.Antialiasing, True)
        
    def sizeHint(self):
        return QSize(300,300)
    
class OutputBox(ImageBox):
    def __init__(self,img=None):
        self.img = img
        self.px = None if img==None else img.toqpixmap()
        super(OutputBox,self).__init__(self.px)
        
    def mouseMoveEvent(self, e):

        if e.buttons() != Qt.RightButton or self.px==None:
            return

        mimeData = QMimeData()
        print("Drag in outputBox")
        mimeData.img = self.img
        drag = QDrag(self)
        drag.setMimeData(mimeData)
        drag.setHotSpot(e.pos() - self.rect().topLeft())

        dropAction = drag.exec_(Qt.MoveAction)
        
    

In [4]:
class GalleryItem(QListWidgetItem):
    def __init__(self,img):
        super(GalleryItem,self).__init__()
        self.img = img
        icon = QIcon(self.img.toqpixmap())
        self.setIcon(icon)
        
        
class Gallery(QListWidget):
    def __init__(self,*args,**kwargs):
        super(Gallery,self).__init__(*args,**kwargs)
        self.setDragDropMode(QAbstractItemView.DragDrop)
        self.setSelectionMode(QAbstractItemView.SingleSelection)
        self.setAcceptDrops(True)
        self.setDragEnabled(True)
        self.setDropIndicatorShown(True)
        self.setIconSize(QSize(80,60))
        
        self.imgs = []

    def addImage(self,img):
        if type(img)==type(""): img = Image.open(img)
        self.imgs.append(img)
        self.addItem(GalleryItem(img))
        
    def dragEnterEvent(self, event):
        print("Drag entring in Gallery...")
        event.accept()
        
    def dragMoveEvent(self,e):
        e.accept()
        
    def dropEvent(self,e):
        img = e.mimeData().img
        self.addImage(img)
        
    def mimeData(self,item):
        data = QMimeData()
        data.img = item[0].img
        return data
    
    def contextMenuEvent(self, event):
        item = self.itemAt(event.pos())
        if item==None: return 
        m = QMenu()
        
        sv = m.addAction("Save")
        sv.triggered.connect(lambda :self.saveFile(item))
        dt = m.addAction("Delete")
        dt.triggered.connect(lambda :self.takeItem(self.row(item)))
        act = m.exec_(self.mapToGlobal(event.pos()))
        
    def saveFile(self,item):
        paths = QFileDialog.getSaveFileName(self,"Save painting")
        print(qd)
        im = item.img
        im.save(paths[0])


In [5]:
class ImageHint(QWidget):

    updated = pyqtSignal(Image.Image)
    
    def __init__(self):
        super(ImageHint,self).__init__()
        self.chosen_points = []     
        self.brushCol = Qt.blue
        self.px = None
        self.img = None
        self.r = 5
        self.setAcceptDrops(True)

    def paintEvent(self,event):
        painter = QPainter(self)
        if self.px==None:
            painter.drawText(QPoint(self.width()//2-5,self.height()//2-5),"No Painting")
            return
        painter.drawPixmap(self.rect(), self.px)
        painter.setRenderHint(QPainter.Antialiasing, True)
        for pos,col,r in self.chosen_points:
            painter.setBrush(QBrush(col))
            qs = self.size()
            sx,sy = qs.width(),qs.height()
            painter.drawEllipse(QPoint(pos[0]*sx,pos[1]*sy),r,r)

        

    def mouseReleaseEvent(self, cursor_event):
        #print("hello:",cursor_event.pos())
        qs = self.size()
        sx,sy = qs.width(),qs.height()
        cur = cursor_event.pos()
        x,y = cur.x(),cur.y()
        rat = (x/sx,y/sy)
        self.chosen_points.append((rat,self.brushCol,self.r))
        self.update()
        self.updated.emit(self.img)
        
    def reset(self):
        #print("Hints reset")
        self.chosen_points = []
        self.update()
        self.updated.emit(self.img)
        
    def undo(self):
        #print("Last hint undone")
        self.chosen_points.pop()
        self.update()
        self.updated.emit(self.img)

    def updateRad(self,r):
        self.r = r
        
    def sizeHint(self):
        return QSize(300,300)
    
    def dragEnterEvent(self, e):
        if hasattr(e.mimeData(),'img'): e.accept()
        else: e.ignore()

    def dropEvent(self, e):
        self.img = e.mimeData().img
        self.px = self.img.toqpixmap()
        self.chosen_points = []
        self.update()
        self.updated.emit(self.img)

In [6]:
duu = {}
class ColorizeTab(QWidget):
    def __init__(self,*args):
        super(ColorizeTab,self).__init__(*args)
        self.hint = ImageHint()
        self.out = OutputBox(Image.open("wwe.jpg"))
        self.setSizePolicy(QSizePolicy(QSizePolicy.Maximum, QSizePolicy.Maximum))
        self.initUI()
    
    def initUI(self):
        mainL = QVBoxLayout()
        self.setLayout(mainL)
        
        self.selCol = QPushButton("select color",self)
        self.selCol.clicked.connect(self.colorChoose)
        
        self.undoBtn = QPushButton("Undo",self)     
        self.undoBtn.clicked.connect(self.hint.undo)

        self.resetBtn = QPushButton("Reset")
        self.resetBtn.clicked.connect(self.hint.reset)
        
        self.curCol = QLabel("CURRENT COLOR",self)
        self.curCol.setAlignment(Qt.AlignCenter)
        self.curCol.setStyleSheet(f"background-color: rgb(0,0,255)")
        self.curCol.setFixedSize(self.selCol.size())

        self.sl = QSlider(Qt.Horizontal)
        self.sl.setMinimum(1)
        self.sl.setMaximum(10)
        self.sl.setValue(5)
        self.sl.setTickPosition(QSlider.TicksBelow)
        self.sl.setTickInterval(1)
        self.sl.valueChanged.connect(self.hint.updateRad)
        
        optL = QHBoxLayout()
        optL.addStretch(1)
        optL.addWidget(self.selCol)
        optL.addStretch(1)
        optL.addWidget(self.curCol)
        optL.addStretch(1)
        optL.addWidget(self.undoBtn)
        optL.addStretch(1)
        optL.addWidget(self.resetBtn)
        optL.addStretch(1)
        optL.addWidget(self.sl)
        optL.addStretch(1)
        
        self.hint.updated.connect(self.colorize)
        
        topL = QHBoxLayout()
        topL.addWidget(self.hint)
        topL.addWidget(self.out)
        
        mainL.addLayout(topL)
        mainL.addLayout(optL)
        
    def colorize(self,img):
        print("Colorizing...")
        self.out.img = img
        self.out.px = img.toqpixmap()
        self.out.update()
        print("Done")
        
    def colorChoose(self):
        colDialog = QColorDialog()
        col = colDialog.getColor()
        self.hint.brushCol = col
        print(type(col))
        self.curCol.setStyleSheet(f"background-color: rgb({col.red()},{col.green()},{col.blue()})")
        self.update()
        
        

In [7]:
class Window(QWidget):
    def __init__(self):
        super(Window,self).__init__()
        self.layout = QVBoxLayout(self)
        
        #-----------------bottom part : Begin---------------
        btmlayout = QHBoxLayout(self)
        
        self.gallery = Gallery(self)
        pdir = "d:/Images/Content/"
        paths = [pdir+f for f in os.listdir(pdir) if f.endswith(".jpg") or f.endswith(".png")]
        for p in paths:
            self.gallery.addImage(p)
        self.gallery.setMaximumWidth(200)
        self.gallery.setDragEnabled(True)
        
        self.colorizer = ColorizeTab(self)
        self.stylizer = QWidget(self)
        layout = QVBoxLayout(self)
        self.pushButton1 = QPushButton("PyQt5 button")
        layout.addWidget(self.pushButton1)
        self.stylizer.setLayout(layout)
        
        self.tabs = QTabWidget(self)
        self.tabs.addTab(self.colorizer,"Colorizer")
        self.tabs.addTab(self.stylizer,"Stylizer")
        
        self.gallery.setSizePolicy(self.tabs.sizePolicy())
        btmlayout.addWidget(self.gallery)
        btmlayout.addWidget(self.tabs)
        
        #--------------bottom part : End---------------
        
        self.title = QLabel("Deep Paint",self)
        self.title.setAlignment(Qt.AlignCenter)
        
        self.layout.addWidget(self.title)
        self.layout.addLayout(btmlayout)
        self.setGeometry(230,250,800,600)

In [8]:
if __name__ == '__main__':

    if not QApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QApplication.instance() 
    w = Window()
    w.show()
    sys.exit(app.exec_())


Drag entring in Gallery...
Colorizing...
Done
Colorizing...
Done


SystemExit: 0

C:\Users\HP\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
c = QColor(255,19,189)

In [10]:
s = [[[c.red(),c.green(),c.blue()]]]
im = np.array(s)/255

In [11]:
im

array([[[1.        , 0.0745098 , 0.74117647]]])

In [12]:
c.red(),c.green(),c.blue()

(255, 19, 189)

In [14]:
a = color.rgb2lab(im)

In [15]:
a[0][0]

array([ 57.46826742,  88.72073436, -28.00670762])

In [ ]:
type(p)

In [ ]:
s = p.toImageFormat()